In [1]:
pip install "pyiceberg[glue]" pyarrow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.9/133.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 8.2 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()  # Upload iceberg_demo_csv.zip from your computer
import zipfile

with zipfile.ZipFile("iceberg_demo_csv.zip", "r") as zip_ref:
    zip_ref.extractall("iceberg_demo_csv")


Saving iceberg_demo_csv.zip to iceberg_demo_csv.zip


In [5]:
!pip install pyarrow pandas
import glob
import pandas as pd

csv_files = sorted(glob.glob('iceberg_demo_csv/events_part_*.csv'))

for p in csv_files:
    df = pd.read_csv(p, dtype={
        'file_no':'int64',
        'id':'int64',
        'name':'string',
        'category':'string',
        'region':'string',
        'value':'int64',
        'price':'float64',
        'is_active':'boolean',
        'ts':'string',
        'event_date':'string',
        'note':'string'
    })
    df['ts'] = pd.to_datetime(df['ts'])
    df['event_date'] = pd.to_datetime(df['event_date']).dt.date

    out = p.replace('.csv', '.parquet')
    df.to_parquet(out, index=False, engine='pyarrow')
    print("Wrote", out)

print("✅ All files converted to Parquet.")


Wrote iceberg_demo_csv/events_part_01.parquet
Wrote iceberg_demo_csv/events_part_02.parquet
Wrote iceberg_demo_csv/events_part_03.parquet
Wrote iceberg_demo_csv/events_part_04.parquet
Wrote iceberg_demo_csv/events_part_05.parquet
Wrote iceberg_demo_csv/events_part_06.parquet
Wrote iceberg_demo_csv/events_part_07.parquet
Wrote iceberg_demo_csv/events_part_08.parquet
Wrote iceberg_demo_csv/events_part_09.parquet
Wrote iceberg_demo_csv/events_part_10.parquet
✅ All files converted to Parquet.


In [6]:
import os
import shutil
import glob

parquet_folder = "iceberg_demo_parquet"
os.makedirs(parquet_folder, exist_ok=True)

# Move all .parquet files from iceberg_demo_csv to the new folder
for file in glob.glob("iceberg_demo_csv/*.parquet"):
    shutil.copy(file, parquet_folder)

print("✅ Parquet files copied to", parquet_folder)

✅ Parquet files copied to iceberg_demo_parquet


In [7]:
import zipfile

zip_filename = "iceberg_demo_parquet.zip"
with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for root, _, files in os.walk(parquet_folder):
        for file in files:
            zipf.write(os.path.join(root, file),
                       arcname=os.path.relpath(os.path.join(root, file), parquet_folder))

print("✅ Zipped parquet folder to", zip_filename)


✅ Zipped parquet folder to iceberg_demo_parquet.zip


In [8]:
from google.colab import files
files.download(zip_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>